# Coleta de dados

Para entender como o bitcoin se comporta ao longo do tempo e realizar a predição de preços, é necessário, pelo menos, de sua série histórica diária. Além disso, alguns fatores podem influenciar no preço da criptomoeda e explicar sua variação, portanto serão estudados como potenciais covariáveis para o modelo.

## Sumário do notebook:

1. Definição do ambiente </br>
    1.1. Bibliotecas </br>
    1.2. Variáveis Globais </br>
    1.3. Funções Úteis </br>

2. Coleta dos dados</br>
    2.1. Bitcoin - Yahoo</br>
    2.2. Outras Criptomoedas + índices Americanos - Yahoo</br>
    2.3. Interesse no Tempo - Pytrends</br>
    2.4. Hash Rate - NASDA</br>

3. Unificação das bases</br>
    3.1. Formato de coluna</br>
    3.2. Correção de datas ausente</br>
    3.3. Salvamento da base no S3</br>

-----------

## 1. Definição do ambiente

### Bibliotecas

In [ ]:
%%capture 

!pip install pandas_datareader
!pip install pytrends
!pip install nasdaq-data-link
!pip install fundingrate

In [2]:
import pandas as pd
import numpy as np

import datetime as dt
from dateutil.relativedelta import *

import pandas_datareader.data as web
from pytrends.request import TrendReq
import nasdaqdatalink
from fundingrate import funding_dydx

In [3]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.expand_frame_repr', True)
# warnings.simplefilter(action='ignore', category=FutureWarning)

### Variáveis globais

In [4]:
#@title parâmetros de formatação dos gráficos {display-mode: "form"}
plotly_kwargs = dict(
    title_x=0.5,
    autosize=False,
    plot_bgcolor='white',
    margin=dict(l=2, r=10, t=30, b=2),
    width=1000,
    height=500,
    hovermode='x unified',
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    )
)

In [5]:
cores_tema = {
   'verde_claro': '#8DD8D3',
   'verde_escuro': '#599191', 
   'azul_petroleo': '#0B6374',
   'cinza_escuro': '#424242',
   'azul_bic': '#27278B', 
    'azul':'#1b86b9',
   'laranja': '#C0791B',  
   'vermelho': '#FD5B58', 
   'rosa': '#D558AB',
   'amarelado': '#D7E6A3'
}

In [6]:
dt_fim = pd.to_datetime('2022-11-17')
periodo = pd.to_datetime([dt_fim + relativedelta(years=-5), dt_fim])

### Funções úteis

In [7]:
def create_corr_plot(series, nlags=None, plot_pacf=False):
    corr_array = pacf(series.dropna(), nlags=nlags, alpha=0.05) if plot_pacf else acf(series.dropna(), nlags=nlags, alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,42])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title, **plotly_kwargs)
    
    return fig

-----------

## 2. Coleta dos dados

**Bases de dados históricas:**

As bases de dados históricas contém registros por data

- `BTC - Bitcoin`: essa é a base principal, que contém o histórico de preços da criptomoeda BTC ao longo do tempo e também o volume de transações.
- `Tether`: preço da Tether (USDT), a maior stablecoin do universo dos ativos digitais, com alta capitalização de mercado - maior do que a grande maioria das criptomoedas.
- `Ether`: preço da Ether (ETH), nome dado ao token nativo do blockchain Ethereum e a segunda maior criptomoeda do mundo em valor de mercado.
- `NDXT`: índice NDXT — que reúne as 100 maiores empresas de tecnologia listada na Nasdaq.
- `SP500`: S&P 500, abreviação de Standard & Poor's 500, é um índice composto por quinhentos ativos cotados nas bolsas de NYSE ou NASDAQ, qualificados devido ao seu tamanho de mercado, sua liquidez e sua representação de grupo industrial.
- `USD`: preço do dólar (referente ao real) no período analisado.
- `Pytrends - btc_iot`: interesse relativo de pesquisas sobre o termo bitcoin no google ao longo do tempo 
- `Hash Rate`: medida usada para calcular a velocidade de resolução de um código da blockchain. O hashrate pode indicar queda na mineração ou fraqueza no mercado, causando pressão descendente no preço


**Variável Target:**

Como queremos predizer o preço de fechamento ajustado da bitcoin, nossa _variável target_ é `Adj Close`, da base BTC. A nomenclatura da variável será alterada para `btc`.

**Dicionário das colunas**:

As bases btc, tether, ether, ndxt e sp500, são extraídas da mesma fonte e possuem as mesmas colunas, descritas abaixo:
1. `Date`: É a a coluna de index, que representa a data associada ao preço daquele registro.
2. `Close`: O preço de fechamento da BTC no respectivo dia, ou sejam o último preço no qual a BTC foi vendida naquele dia.
3. `Open`: O preço de abertura da BTC no respectivo dia, ou sejam o primeiro preço no qual a BTC foi vendida naquele dia.
4. `High`: O maior preço que a BTC foi vendida no respectivo dia.
5. `Low`:  O menor preço que a BTC foi vendida no respectivo dia.
6. `Volume`: O número de transações totais naquele dia.
7. `Adj Close`: O preço de fechamento ajustado por dividendos e desdobramentos de ações.

A coluna `Adj Close`, representando o preço de fechameto ajustado, será a coluna priorizada para todas as variáveis. 

As demais bases de dados contém somente uma coluna referente à informação descrita no tópico de "bases de dados históricas", portanto, não serão repetidos aqui.

### 2.1. Bitcoin - Yahoo

In [8]:
btc = web.DataReader('BTC-USD', data_source='yahoo', start=periodo[0], end=periodo[1])

# Ajustando nomes das colunas
btc.columns = ['btc_' + c.lower() for c in btc.columns]
btc = btc.rename(columns={'btc_adj close':'btc'})

In [9]:
btc.shape

(1828, 6)

In [10]:
btc.head(3)

,btc_high,btc_low,btc_open,btc_close,btc_volume,btc
Date,,,,,,
2017-11-17,8004.589844,7561.089844,7853.569824,7708.990234,4651670016,7708.990234
2017-11-18,7884.990234,7463.439941,7697.209961,7790.149902,3667190016,7790.149902
2017-11-19,8101.910156,7694.100098,7766.029785,8036.490234,3149319936,8036.490234


In [11]:
btc.tail(3)

,btc_high,btc_low,btc_open,btc_close,btc_volume,btc
Date,,,,,,
2022-11-16,16960.294922,16430.111328,16884.341797,16669.439453,33925512989,16669.439453
2022-11-17,16726.439453,16460.683594,16670.425781,16687.517578,27868914022,16687.517578
2022-11-18,16947.058594,16564.611328,16687.912109,16697.777344,26862218609,16697.777344


A base histórica de bitcoins tem 1828 registros e 6 colunas, com seu histórico completo no período selecionado.

### 2.2. Yahoo - Outras criptomoedas + Indices Americanos

In [12]:
tickers = ['USDT-USD', 'ETH-USD'] + ["^NDXT", '^GSPC', "USDBRL=x"] # criptos + indices
ticker_names = ['usdt','eth', 'ndxt', 'sp500', 'usd']

In [13]:
df_yahoo = web.get_data_yahoo(tickers, start=periodo[0], end=periodo[1])["Adj Close"]
df_yahoo.columns=ticker_names

In [14]:
df_yahoo.head(3)

,usdt,eth,ndxt,sp500,usd
Date,,,,,
2017-11-17,1.00139,332.394012,4031.629883,2578.850098,3.2706
2017-11-18,1.00231,347.612000,NaN,NaN,NaN
2017-11-19,1.00244,354.385986,NaN,NaN,NaN


In [15]:
df_yahoo.tail(3)

,usdt,eth,ndxt,sp500,usd
Date,,,,,
2022-11-17,0.999402,1200.808594,6078.709961,3946.560059,5.3990
2022-11-18,0.999163,1212.300293,NaN,NaN,5.4221
2022-11-19,NaN,NaN,NaN,NaN,5.3827


Para essa base, o dado é diário mas já observamos coletas faltantes que devem ser tratadas posteriormente.
Observa-se também que o filtro de data final da função não funciona muito bem, mas o caso será tratado posteriormente para todas as bases, pois o problema está na função (outros testes foram feitos).

In [16]:
df_yahoo.describe()

,usdt,eth,ndxt,sp500,usd
count,1828.000000,1828.000000,1259.000000,1259.000000,1306.000000
mean,1.001591,1131.236136,6021.373109,3420.030182,4.613303
std,0.005766,1203.097282,1748.625487,675.156441,0.796489
min,0.966644,84.308296,3457.919922,2237.399902,3.145100
25%,0.999982,209.034111,4393.555176,2821.954956,3.865525
50%,1.000500,478.433487,5643.189941,3224.729980,4.932600
75%,1.002497,1802.185455,7543.064941,4018.845093,5.323050
max,1.077880,4812.087402,9855.419922,4796.560059,5.886400


### 2.3. Pytrends - interesse ao longo do tempo

In [17]:
# Conexão com google
bittrends = TrendReq(hl='en-US', tz=360) 

# Puxando dados
bittrends.build_payload(['bitcoin'], cat=0, timeframe=" ".join(periodo.astype(str)), geo='') 
btc_iot = bittrends.interest_over_time() 

# Filtro 
btc_iot = btc_iot[~btc_iot['isPartial']]

# Ajustando colunas
btc_iot.index.names=['Date']
btc_iot = btc_iot.rename(columns={'bitcoin':'iot'}).drop(columns=['isPartial'])

In [18]:
btc_iot.head(3)

,iot
Date,
2017-11-19,28
2017-11-26,66
2017-12-03,89


In [19]:
btc_iot.tail(3)

,iot
Date,
2022-10-23,13
2022-10-30,14
2022-11-06,26


A base de interesses de pesquisa é semanal, portanto deverá ser preenchida seguindo alguma estratégia para ter periodicidade diária. Também percebe-se que a atualização é mais demorada desses dados, apesar da data máxima ser 17/11, a última atualização é 06/11, o que pode prejudicar uma previsão diária.

### 2.4. Hash rate

Fonte: https://data.nasdaq.com/

In [20]:
hash_rate = nasdaqdatalink.get('BCHAIN/HRATE', start_date=periodo[0], end_date=periodo[1])
hash_rate = hash_rate.rename(columns={'Value':'hash_rate'})

In [21]:
hash_rate.head(3)

,hash_rate
Date,
2017-11-17,1.132691e+07
2017-11-18,9.224307e+06
2017-11-19,1.085213e+07


In [22]:
hash_rate.tail(3)

,hash_rate
Date,
2022-11-15,2.832561e+08
2022-11-16,2.741188e+08
2022-11-17,2.521893e+08


Os dados parecem ser estruturados e estarem corretos.

--------------

## 3. Unificação das bases

In [23]:
df_list = [btc, df_yahoo, btc_iot, hash_rate]

Ajuste de index para o join

In [24]:
df_concat = []
for df in df_list:
    # Ajuste de index para o join
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    
    # Filtro de datas
    df = df.loc[(df.index>=periodo[0]) & (df.index<=periodo[1])]
    
    df_concat.append(df)

In [25]:
df_all = pd.concat(df_concat, axis=1)

In [26]:
df_all.head(3)

,btc_high,btc_low,btc_open,btc_close,btc_volume,btc,usdt,eth,ndxt,sp500,usd,iot,hash_rate
Date,,,,,,,,,,,,,
2017-11-17,8004.589844,7561.089844,7853.569824,7708.990234,4651670016,7708.990234,1.00139,332.394012,4031.629883,2578.850098,3.2706,NaN,1.132691e+07
2017-11-18,7884.990234,7463.439941,7697.209961,7790.149902,3667190016,7790.149902,1.00231,347.612000,NaN,NaN,NaN,NaN,9.224307e+06
2017-11-19,8101.910156,7694.100098,7766.029785,8036.490234,3149319936,8036.490234,1.00244,354.385986,NaN,NaN,NaN,28.0,1.085213e+07


In [27]:
df_all.tail(3)

,btc_high,btc_low,btc_open,btc_close,btc_volume,btc,usdt,eth,ndxt,sp500,usd,iot,hash_rate
Date,,,,,,,,,,,,,
2022-11-15,17051.962891,16542.550781,16617.484375,16884.613281,36599436183,16884.613281,0.999130,1251.736206,6344.140137,3991.729980,5.3312,NaN,2.832561e+08
2022-11-16,16960.294922,16430.111328,16884.341797,16669.439453,33925512989,16669.439453,0.999229,1215.602539,6121.200195,3958.790039,5.3317,NaN,2.741188e+08
2022-11-17,16726.439453,16460.683594,16670.425781,16687.517578,27868914022,16687.517578,0.999402,1200.808594,6078.709961,3946.560059,5.3990,NaN,2.521893e+08


Essa base será utilizada para estudos e modelagem, portanto, será salva no S3, no bucket raw, que serão os dados antes de realizar feature engineering e correções.

### 3.1. Formato de colunas

In [38]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2017-11-17 to 2022-11-17
Freq: D
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   btc_high    1827 non-null   float64
 1   btc_low     1827 non-null   float64
 2   btc_open    1827 non-null   float64
 3   btc_close   1827 non-null   float64
 4   btc_volume  1827 non-null   int64  
 5   btc         1827 non-null   float64
 6   usdt        1827 non-null   float64
 7   eth         1827 non-null   float64
 8   ndxt        1259 non-null   float64
 9   sp500       1259 non-null   float64
 10  usd         1304 non-null   float64
 11  iot         260 non-null    float64
 12  hash_rate   1827 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 264.4 KB


Conferência: Todas as colunas já são numéricas. A coluna de IOT, como esperado, é a que mais possui valores nulos, pois sua frequência é semanal, enquanto das demais é diária.

### 3.2. Correção de datas ausentes

Para ter certeza que não há datas faltando no dataframe, foi realizado o _reindex_ do dataframe de acordo com o intervalo de datas esperado.

In [28]:
print(df_all.shape)

# Para ter certeza que não há buracos no dataframe série
df_all = df_all.apply(lambda group: group.reindex(pd.date_range(periodo[0], periodo[1], freq='D'), 
                                                  #method='nearest' - vou deixar o método default: none, para que não haja preenchimetnos inadequados
                                                  )).sort_index()

print(df_all.shape)

(1827, 13)
(1827, 13)


In [29]:
df_all.describe()

,btc_high,btc_low,btc_open,btc_close,btc_volume,btc,usdt,eth,ndxt,sp500,usd,iot,hash_rate
count,1827.000000,1827.000000,1827.000000,1827.000000,1.827000e+03,1827.000000,1827.000000,1827.000000,1259.000000,1259.000000,1304.000000,260.000000,1.827000e+03
mean,20760.603587,19665.947578,20246.906348,20249.268044,2.637458e+10,20249.268044,1.001593,1131.191765,6021.373109,3420.030182,4.612093,21.296154,1.116821e+08
std,17464.880867,16512.579105,17024.947416,17018.145277,1.993908e+10,17018.145277,0.005768,1203.425176,1748.625487,675.156441,0.796499,15.437781,6.609720e+07
min,3275.377930,3191.303467,3236.274658,3236.761719,2.923670e+09,3236.761719,0.966644,84.308296,3457.919922,2237.399902,3.145100,6.000000,8.342572e+06
25%,7900.539551,7511.864746,7688.712402,7684.071777,1.108360e+10,7684.071777,0.999983,208.977219,4393.555176,2821.954956,3.865350,11.000000,4.857148e+07
50%,11089.799805,10516.417969,10818.156250,10821.726562,2.410343e+10,10821.726562,1.000500,477.493988,5643.189941,3224.729980,4.927400,16.000000,1.096787e+08
75%,34778.076172,32265.797852,33767.375000,33734.515625,3.577598e+10,33734.515625,1.002497,1802.761414,7543.064941,4018.845093,5.321700,27.000000,1.573418e+08
max,68789.625000,66382.062500,67549.734375,67566.828125,3.509679e+11,67566.828125,1.077880,4812.087402,9855.419922,4796.560059,5.886400,100.000000,3.167823e+08


Não havia buracos no dataframe concatenado, pois não houve mudança de tamanho.

Agora toda a base está filtrada no período correto.

### 3.2. Salvamento da base no S3

Para o salvamento da base no s3, é preciso realizar a conexão com o bucket de destino.

In [30]:
import os, io, boto3

In [35]:
s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(bucket, folder, filename, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=True, index=True)
    s3_resource.Bucket(bucket).Object(os.path.join(folder, filename)).put(Body=csv_buffer.getvalue())

In [36]:
upload_s3_csv(bucket = 'projetointerdisciplinars2'
              , folder = 'data/raw'
              , filename = 'df_btc_covars.csv'
              , dataframe = df_all
              , )

In [33]:
%%capture
!pip install --upgrade pip
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader
!pip install cmasher
!pip install pytrends